In [ ]:
import init_notebook

In [ ]:

import numpy as np
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Conv1D, BatchNormalization, Dense, Flatten, Dropout, InputLayer, Input
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from visualizations.evaluation import plot_confusion_matrix, plot_classification_metric_summary
from evaluation.model_evaluation import get_classification_metric_summary


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


#######################################################################################

x_train = x_train/255
x_test = x_test/255

y_train = keras.utils.to_categorical(y_train, num_classes=10)



i_layer = Input(shape=(28,28))

x = Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(i_layer)
x = BatchNormalization(momentum=0.9)(x)
x = Dropout(0.2)(x)

x = Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(x)
x = BatchNormalization(momentum=0.9)(x)
x = Dropout(0.2)(x)

x = Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu')(x)
x = BatchNormalization(momentum=0.9)(x)
x = Dropout(0.2)(x)

x = Flatten()(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.2)(x)

o_layer = Dense(units=10, activation='softmax')(x)

model = Model(i_layer, o_layer)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# Lists to store training and validation loss values
train_loss_list = []
val_loss_list = []

# Training loop
for epoch in range(10):
    history = model.fit(x_train, y_train, batch_size=1000, epochs=1, validation_split=0.2, verbose=2)

    # Append training and validation loss to the lists
    train_loss_list.append(history.history['loss'][0])
    val_loss_list.append(history.history['val_loss'][0])

# Plot Training-Validation curve
plt.plot(range(1, 11), train_loss_list, label='Training Loss')
plt.plot(range(1, 11), val_loss_list, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training-Validation Curve')
plt.legend()
plt.show()

#######################################################################################

In [ ]:
y_test_pred_class = model.predict(x_test)
y_test_pred_class = np.argmax(y_test_pred_class, axis=-1)
ac_score = accuracy_score(y_test, y_test_pred_class)



plot_confusion_matrix(y_test, y_test_pred_class,"Confusion matrix for MNIST dataset using CNNs" )

class_metrics, accuracy = get_classification_metric_summary(y_test, y_test_pred_class)
plot_classification_metric_summary(class_metrics, f"MNIST Class Classification Metrics for Test Set (MLP) (Accuracy = {accuracy:.2f}) ")